In [ ]:
import os
import h5py
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.signal import resample
from torch.utils.data import DataLoader, TensorDataset
from torch import optim
import torchmetrics
from torch.utils.data import random_split
from ray import tune
from ray import train
from ray import cloudpickle as cloudpickle
import pickle
from ray.train import Checkpoint
from scipy import stats

### Settings

In [ ]:
label_mapping = {'rest': 0, 'motor': 1, 'story': 2, 'memory': 3}
downsample_rate = 35595 * (1 - 0.1)
batch_size = 4
epochs = 5

### Importing and Preparing Data

In [ ]:
def get_dataset_name(file_name_with_dir) :
    filename_without_dir = file_name_with_dir.split('/')[-1]
    temp = filename_without_dir.split('_')[:-1]
    dataset_name = "_".join(temp)
    return dataset_name

def get_dataset_from_dir(dir_name, label_mapper, downsample_rate=0.0, scaling='standard', use_skorch=False):
    data_list = []
    labels_list = []
    for filename in os.listdir(dir_name):
        filename_path = dir_name + '/' + filename
        with h5py.File(filename_path, 'r') as f:
            dataset_name = get_dataset_name(filename_path)
            data = f.get(dataset_name)
            data = np.array(data)
            if 'rest' in filename:
                labels_list.append('rest')
            elif 'motor' in filename:
                labels_list.append('motor')
            elif 'story' in filename:
                labels_list.append('story')
            elif 'memory' in filename:
                labels_list.append('memory')
            else:
                raise ValueError(f'Inappropriate filename: {dir_name}/{filename}')

        # scaling
        if scaling == 'standard':
            scaler = StandardScaler()
            data_scaled = scaler.fit_transform(data) # NB each h5 file is scaled seperately
        elif scaling =='minmax':
            scaler = MinMaxScaler()
            data_scaled = scaler.fit_transform(data)
        else:
            data_scaled = data
        # downsampling
        if downsample_rate > 0:
            steps_after_downsampling = int(35595 * (1 - downsample_rate))
            data_downsampled = resample(data_scaled, num=steps_after_downsampling, axis=1)
        else:
            data_downsampled = data_scaled
        data_transposed = np.transpose(data_downsampled) # rows: timesteps, columns: sensors
        data_list.append(data_transposed)
    features = np.stack(data_list)
    labels = np.array([label_mapper[label] for label in labels_list])
    if use_skorch:
        return features, labels
    
    # Convert data arrays to PyTorch tensors
    X_tensor = torch.FloatTensor(features) # Convert training data to PyTorch tensor
    y_tensor = torch.LongTensor(labels) # Convert testing labels to PyTorch tensor

    # Create PyTorch datasets using tensors
    dataset = TensorDataset(X_tensor, y_tensor)  # Create dataset for training
    return dataset

### Model definition

In [ ]:
class MEG_LSTM_Bidirectional(torch.nn.Module):
    def __init__(self, hidden_size=64, num_layers=2, dropout_prob=0.1):
        super(MEG_LSTM_Bidirectional, self).__init__()
        self.input_size = 248 # input size is data from 248 sensors, not configurable
        self.output_size = 4 # output size is 4 classes, not configurable
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(248, self.hidden_size, self.num_layers, dropout=dropout_prob, bidirectional=True, batch_first=True) # input size is 248, not configurable
        self.dropout = torch.nn.Dropout(p=dropout_prob)
        self.fc = torch.nn.Linear(hidden_size * 2, 4) # output size is 4, not configurable

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))  # Passing input through LSTM
        out = self.dropout(out)
        # Get output from the last time step
        out = self.fc(out[:, -1, :])
        return out

class MEG_LSTM(torch.nn.Module):
    def __init__(self, hidden_size=64, num_layers=2, dropout_prob=0.1):
        super(MEG_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(248, self.hidden_size, self.num_layers, bidirectional=True, batch_first=True) # input size is 248, not configurable
        self.dropout = torch.nn.Dropout(p=dropout_prob)
        self.fc = torch.nn.Linear(hidden_size, 4) # output size is 4, not configurable

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))  # Passing input through LSTM
        
        # Get output from the last time step
        out = self.fc(out[:, -1, :])
        return out
    
class MEG_LSTM_Complex(torch.nn.Module):
    def __init__(self, hidden_size=64, num_layers=2, dropout_prob=0.1):
        super(MEG_LSTM_Complex, self).__init__()
        self.input_size = 248
        self.output_size = 4
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm1 = torch.nn.LSTM(self.input_size, self.hidden_size, self.num_layers, dropout=dropout_prob, bidirectional=True, batch_first=True)
        self.tanh1 = torch.nn.Tanh()
        # Batch Normalization
        self.batch_norm1 = torch.nn.BatchNorm1d(self.hidden_size * 2)
        self.lstm2 = torch.nn.LSTM(self.hidden_size * 2, self.hidden_size, self.num_layers, dropout=dropout_prob, bidirectional=True, batch_first=True)
        self.tanh2 = torch.nn.Tanh()
        self.batch_norm2 = torch.nn.BatchNorm1d(self.hidden_size * 2)
        self.dense = torch.nn.Linear(hidden_size * 2, hidden_size * 2)
        # ReLU activation
        self.elu = torch.nn.ELU()
        # Output layer
        self.fc = torch.nn.Linear(hidden_size * 2, self.output_size)


    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.tanh1(out)
        out = self.batch_norm1(out.permute(0, 2, 1)).permute(0, 2, 1)
        out, _ = self.lstm2(out)
        out = self.tanh2(out)
        out = self.batch_norm2(out.permute(0, 2, 1)).permute(0, 2, 1)
        out = self.dense(out[:, -1, :])
        out = self.elu(out)
        out = self.fc(out)
        return out


class MEG_GRU_Complex(torch.nn.Module):
    def __init__(self, hidden_size, num_layers, dropout_prob):
        super(MEG_GRU_Complex, self).__init__()
        self.input_size = 248
        self.output_size = 4
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.gru1 = torch.nn.GRU(self.input_size, self.hidden_size, self.num_layers, dropout=dropout_prob, bidirectional=True, batch_first=True)
        self.tanh1 = torch.nn.Tanh()
        # Batch Normalization
        self.batch_norm1 = torch.nn.BatchNorm1d(self.hidden_size * 2)
        self.gru2 = torch.nn.GRU(self.hidden_size * 2, self.hidden_size, self.num_layers, dropout=dropout_prob, bidirectional=True, batch_first=True)
        self.tanh2 = torch.nn.Tanh()
        self.batch_norm2 = torch.nn.BatchNorm1d(self.hidden_size * 2)
        self.dense = torch.nn.Linear(hidden_size * 2, hidden_size * 2)
        # ReLU activation
        self.elu = torch.nn.ELU()
        # Output layer
        self.fc = torch.nn.Linear(hidden_size * 2, self.output_size)


    def forward(self, x):
        out, _ = self.gru1(x)
        out = self.tanh1(out)
        out = self.batch_norm1(out.permute(0, 2, 1)).permute(0, 2, 1)
        out, _ = self.gru2(out)
        out = self.tanh2(out)
        out = self.batch_norm2(out.permute(0, 2, 1)).permute(0, 2, 1)
        out = self.dense(out[:, -1, :])
        out = self.elu(out)
        out = self.fc(out)
        return out


#### Training and tuning

In [ ]:
# Early stopping mechanism to speed up training
class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.min_val_loss = float('inf')
    
    def stop_training(self, val_loss) -> bool:
        """Assess whether early stopping is necessary through validation loss."""
        if val_loss < self.min_val_loss:
            self.min_val_loss = val_loss
            self.counter = 0
        elif val_loss > self.min_val_loss:
            self.counter += 1
            if self.counter > self.patience:
                return True
        return False

In [ ]:
def train_meg(config, trainset, cwd):
    net = MEG_GRU_Complex(hidden_size=config["hidden_size"], num_layers=config["nr_layers"], dropout_prob=config["dropout_rate"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
    net.to(device)

    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=config["lr"])

    # initialize train- and testloader
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(trainset, [test_abs, len(trainset) - test_abs])

    trainloader = DataLoader(train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8)
    valloader = DataLoader(val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8)

    # initialize early stopping
    stopper = EarlyStopping(patience=3)

    # start training loop
    for epoch in range(0, config["max_epochs"]):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # data is a list with shape [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward step
            outputs = net(inputs)
            loss = loss_function(outputs, labels)
            
            # backward step
            loss.backward()
            
            # optimizer step
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 5 == 4: # return statistics every 5 minibatches
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / epoch_steps}")
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = loss_function(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Store a checkpoint as dict:
        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }

        with open(os.path.join(cwd, 'Checkpoints/checkpoint_data.pkl'), 'wb') as fp:
            cloudpickle.dump(checkpoint_data, fp)

        checkpoint = Checkpoint.from_directory(os.path.join(cwd, 'Checkpoints'))
        train.report({"loss": val_loss / val_steps, "accuracy": correct / total}, checkpoint=checkpoint)

        # check whether early stopping is necessary
        if stopper.stop_training(val_loss):
            print(f'stopped early at epoch {epoch}')
            break

    print("Finished Training")

def eval_model(net, testset, device="cpu"):

    # initialize test loader
    testloader = DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )
    result_list = []
    correct = 0
    total = 0

    # calculate statistics
    metric1 = torchmetrics.Accuracy(task="multiclass", num_classes=4)
    metric2 = torchmetrics.Precision(task="multiclass", average="macro", num_classes=4)
    metric3 = torchmetrics.Recall(task="multiclass", average="macro", num_classes=4)
    metric4 = torchmetrics.F1Score(task="multiclass", average="macro", num_classes=4)
    with torch.no_grad():
        i = 0
        for data in testloader:
            features, labels = data
            features, labels = features.to(device), labels.to(device)
            outputs = net(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            acc = metric1(predicted, labels)
            pre = metric2(predicted, labels)
            rec = metric3(predicted, labels)
            f1 = metric4(predicted, labels)
            for pred, lab in zip(predicted, labels):
                if pred == lab:
                    result_list.append(1)
                else:
                    result_list.append(0)
            i += 1
    acc = metric1.compute()
    pre = metric2.compute()
    rec = metric3.compute()
    f1 = metric4.compute()
    print(f"Accuracy {i}: {acc}")
    print(f"Precision {i}: {pre}")
    print(f"Recall {i}: {rec}")
    print(f"F1 {i}: {f1}")
    return[acc, pre, rec, f1, result_list]

In [ ]:
def main(train_dir: str, downsample_to: float, num_samples: int):
    print(f'Using downsampling rate: {downsample_to}.')
    cwd = os.getcwd()
    label_mapping = {'rest': 0, 'motor': 1, 'story': 2, 'memory': 3}
    
    train_set = get_dataset_from_dir(dir_name=train_dir, label_mapper=label_mapping, downsample_rate=downsample_to, scaling="standard")

    config = {
            "lr": tune.choice([1e-4, 1e-3, 1e-2]), # 3
            "hidden_size": tune.choice([32, 64, 128]), # 3
            "nr_layers": tune.choice([2, 3, 4]), # 3
            "batch_size": tune.choice([4, 8, 16]), # 3
            "max_epochs": tune.choice([8]), # 1
            "dropout_rate": tune.choice([0.1, 0.25, 0.5]) # 3
    }

    result = tune.run(
        tune.with_parameters(train_meg, trainset=train_set, cwd=cwd),
        resources_per_trial={"cpu": 11, "gpu": 0},
        config=config,
        num_samples=num_samples,
    )

    print('getting best trial')
    best_trial = result.get_best_trial("loss", "min", "last")
    if best_trial is None:
        return
    else:
        print(f"Best val_loss: {best_trial.last_result['loss']}")
        print(f"Best accuracy: {best_trial.last_result['accuracy']}")
        print(f"Best config: {best_trial.config}")

        print('initializing best model so far')
        best_trained_model = MEG_GRU_Complex(best_trial.config["hidden_size"], best_trial.config["nr_layers"], best_trial.config["dropout_rate"])
        device = "cpu"
        if torch.cuda.is_available():
            device = "cuda:0"
        best_trained_model.to(device)
        
        best_checkpoint = best_trial.checkpoint

        # load checkpoint
        if best_checkpoint is not None:
            with best_checkpoint.as_directory() as checkpoint_dir:
                with open(os.path.join(checkpoint_dir, 'checkpoint_data.pkl'), 'rb') as fp:
                    checkpoint = cloudpickle.load(fp)

            best_trained_model.load_state_dict(checkpoint["net_state_dict"])

            test_set1 = get_dataset_from_dir(dir_name="Data/Cross/test1/", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")
            test_set2 = get_dataset_from_dir(dir_name="Data/Cross/test2/", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")
            test_set3 = get_dataset_from_dir(dir_name="Data/Cross/test3/", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")

            test_acc1 = eval_model(best_trained_model, test_set1)
            test_acc2 = eval_model(best_trained_model, test_set2)
            test_acc3 = eval_model(best_trained_model, test_set3)
            print("Best trial test set accuracy: {}".format(test_acc1 + test_acc2 + test_acc3))
            with open(os.path.join(cwd, f'Models/{best_trained_model._get_name()}'), 'wb') as model_file:
                pickle.dump(best_trained_model, model_file)

main(train_dir='Data/Cross/train/', downsample_to=0.9, num_samples=60)

### Model evaluation

In [ ]:
# Enter the hidden size an number of layers of the checkpoint
best_trained_model = MEG_LSTM_Complex(128, 3, 0.1)

# Move the best checkpoint into the checkpoint folder
with open(os.path.join("Checkpoints", 'checkpoint_lstm_intra.pkl'), 'rb') as fp:
    checkpoint = cloudpickle.load(fp)

best_trained_model.load_state_dict(checkpoint["net_state_dict"])

test_set1 = get_dataset_from_dir(dir_name="Data/intra/test", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")
# test_set2 = get_dataset_from_dir(dir_name="Data/Cross/test2/", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")
# test_set3 = get_dataset_from_dir(dir_name="Data/Cross/test3/", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")

a1 = eval_model(best_trained_model, test_set1)
# a2 = eval_model(best_trained_model, test_set2)
# a3 = eval_model(best_trained_model, test_set3)
# print("Best trial test set accuracy: {}".format((a1[0] + a2[0] + a3[0]) / 3))
# print("Best trial test set precision: {}".format((a1[1] + a2[1] + a3[1]) / 3))
# print("Best trial test set accuracy: {}".format((a1[2] + a2[2] + a3[2]) / 3))
# print("Best trial test set accuracy: {}".format((a1[3] + a2[3] + a3[3]) / 3))



In [ ]:
# Enter the hidden size an number of layers of the checkpoint
model1 = MEG_LSTM_Complex(128, 3, 0.1)
model2 = MEG_GRU_Complex(128, 3, 0.1)


# Move the best checkpoint into the checkpoint folder
with open(os.path.join("Checkpoints", 'checkpoint_lstm_intra.pkl'), 'rb') as fp:
    checkpoint1 = cloudpickle.load(fp)

with open(os.path.join("Checkpoints", 'checkpoint_gru_intra.pkl'), 'rb') as fp:
    checkpoint2 = cloudpickle.load(fp)

model1.load_state_dict(checkpoint1["net_state_dict"])
model2.load_state_dict(checkpoint2["net_state_dict"])


test_set1 = get_dataset_from_dir(dir_name="Data/intra/test", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")
# test_set2 = get_dataset_from_dir(dir_name="Data/Cross/test2/", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")
# test_set3 = get_dataset_from_dir(dir_name="Data/Cross/test3/", label_mapper=label_mapping, downsample_rate=0.9, scaling="standard")

e1 = eval_model(model1, test_set1)
e2 = eval_model(model2, test_set1)

In [ ]:
# prepare arrays to use for statistical tests
lstm_correct = e1[4]
gru_correct = e2[4]

def perform_mcnemar_test(model1_prediction_vals, model2_prediction_vals):
    # List which contain data entries in which both the models predicted correctly 
    both = sum([logit_val and svm_val for logit_val, svm_val in zip(lstm_correct,gru_correct)])
    # List which contain data entries in which only model 1 predicted correctly 
    model1 = sum([logit_val and not svm_val for logit_val, svm_val in zip(lstm_correct, gru_correct)])
    # List which contain data entries in which only model 2 predicted correctly 
    model2 = sum([not logit_val and svm_val for logit_val, svm_val in zip(lstm_correct, gru_correct)])
    # List which contain data entries in which both the models predicted incorrectly 
    neither = sum([not logit_val and not svm_val for logit_val, svm_val in zip(lstm_correct, gru_correct)])

    # Merge these lists into a matrix for the Mcnemar test 
    contingencies = [[both, model1],
                     [model2, neither]]

    # Perform Mcnemar test with the given contingencies 
    mcnemar_statistic = ((model1 - model2)**2)/(model1 + model2)
    mcnemar_pvalue = stats.chi2.sf(mcnemar_statistic, 1)
    print('McNemar\'s test to verify significant difference between the models:\n', 'Statistic: %.3f' % (mcnemar_statistic), 'p-value:', mcnemar_pvalue)

perform_mcnemar_test(lstm_correct, gru_correct)